In [117]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import datetime
from sklearn.pipeline import FeatureUnion

# Split training and testing dataset

In [118]:
data=pd.read_pickle('data.pkl')

In [119]:
train=data[data.index<=datetime.datetime(2015, 5, 4)]
test=data[data.index>datetime.datetime(2015, 5, 4)]

In [120]:
train.head()

,CPI,NOS,PCR,OIL,SI,BDI,resid,IC,TERM,VRP
AsOfDate,,,,,,,,,,
1990-06-08,0.046737,0.016941,-1.622923,NaN,NaN,1278.0,-0.013576,NaN,0.50,7.197727
1990-06-11,0.046737,0.016941,-1.615366,NaN,NaN,1264.0,-0.013576,NaN,0.49,5.607431
1990-06-12,0.046737,0.016941,-1.618868,NaN,NaN,1254.0,-0.013576,NaN,0.49,6.403252
1990-06-13,0.046737,0.016941,-1.625316,NaN,NaN,1243.0,-0.013576,NaN,0.47,7.793147
1990-06-14,0.046737,0.016941,-1.620454,NaN,NaN,1232.0,-0.013576,NaN,0.46,7.665179


# Prepocess

In [121]:
#pipelines to select columns
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]
#de-median
class DeMedian(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        self.median=0
    def fit(self, X):
        self.median=X.median()
        return self  # nothing else to do
    def transform(self, X):
        return X-self.median
#take log
class Log(BaseEstimator, TransformerMixin):
    def __init__(self): 
        return
    def fit(self, X):
        return self
    def transform(self, X):
        return np.log(X)

In [122]:
demedian_list=['VRP','IC']
log_median_list=['BDI']
other_list=[i for i in train.columns if i not in demedian_list+log_median_list]
demedian_pipeline = Pipeline([
        ('selector', DataFrameSelector(demedian_list)),
        ('demedian', DeMedian()),
    ])

log_median_pipeline = Pipeline([
        ('selector', DataFrameSelector(log_median_list)),
        ('log', Log()),
        ('demedian', DeMedian()),
    ])
other_pipeline=Pipeline([
    ('selector', DataFrameSelector(other_list)),
])
full_pipeline = FeatureUnion(transformer_list=[
        ("demedian_pipline", demedian_pipeline),
        ("log_median_pipeline", log_median_pipeline),
        ("other_pipeline", other_pipeline),
        
    ])

In [123]:
train_prepared=full_pipeline.fit_transform(train)

In [124]:
train_prepared

array([[ 0.73708797,         nan, -0.1528084 , ...,         nan,
        -0.01357599,  0.5       ],
       [-0.85320834,         nan, -0.16382346, ...,         nan,
        -0.01357599,  0.49      ],
       [-0.0573871 ,         nan, -0.17176631, ...,         nan,
        -0.01357599,  0.49      ],
       ...,
       [-0.64222115, -4.3       , -0.92404402, ...,  0.98136784,
        -0.01269891,  2.04      ],
       [-2.57190703, -6.98      , -0.93083521, ...,  0.53942159,
        -0.01269891,  2.11      ],
       [-2.33401527, -5.21      , -0.93083521, ...,  0.48065865,
        -0.01269891,  2.14      ]])

# Tables

In [125]:
df_prepared=pd.DataFrame(train_prepared,columns=demedian_list+log_median_list+other_list)

## Table1

In [136]:
summary=df_prepared.describe()
summary.loc['Stdev']=df_prepared.std()
summary.loc['Skewness']=df_prepared.skew()
summary.loc['Kurtosis']=df_prepared.kurt()
summary

,VRP,IC,BDI,CPI,NOS,PCR,OIL,SI,resid,TERM
count,6275.000000,1841.000000,6275.000000,6275.000000,6275.000000,6275.000000,6101.000000,499.000000,6275.000000,6272.000000
mean,0.430350,-0.450951,0.157520,0.025519,-0.016805,-1.361110,0.239927,0.537009,0.005230,1.865515
std,3.458404,10.938760,0.617765,0.012576,0.038633,0.363751,0.390506,0.234051,0.025816,1.161928
min,-24.511136,-39.680000,-1.073412,-0.019588,-0.181167,-2.260664,-0.818794,0.105541,-0.052577,-0.950000
25%,-1.740792,-7.350000,-0.278546,0.017475,-0.039262,-1.609558,0.000371,0.378742,-0.009389,0.910000
50%,0.000000,0.000000,0.000000,0.026211,-0.017843,-1.412000,0.211595,0.493194,0.004489,2.000000
75%,2.131014,5.980000,0.461827,0.031899,0.002775,-1.140890,0.510608,0.632710,0.018684,2.790000
max,23.839293,46.330000,2.069401,0.063796,0.196459,-0.333100,1.172076,1.636367,0.091062,3.870000
Stdev,3.458404,10.938760,0.617765,0.012576,0.038633,0.363751,0.390506,0.234051,0.025816,1.161928
Skewness,0.703677,0.137750,0.950532,-0.162354,0.143467,0.648741,0.027659,1.497838,0.601011,-0.290813


## Table2

In [128]:
df_prepared.corr()

,VRP,IC,BDI,CPI,NOS,PCR,OIL,SI,resid,TERM
VRP,1.000000,0.351058,-0.108147,-0.098929,-0.209388,0.031323,-0.174315,0.454565,0.075844,0.092356
IC,0.351058,1.000000,-0.409923,-0.162414,-0.235873,0.040777,-0.430414,0.061569,0.341942,0.398803
BDI,-0.108147,-0.409923,1.000000,0.249523,0.224709,-0.442579,0.466450,-0.052131,0.302608,-0.071969
CPI,-0.098929,-0.162414,0.249523,1.000000,0.390285,-0.534570,0.396217,0.004746,-0.324996,-0.238530
NOS,-0.209388,-0.235873,0.224709,0.390285,1.000000,-0.141374,0.372830,0.024798,-0.161129,-0.238081
PCR,0.031323,0.040777,-0.442579,-0.534570,-0.141374,1.000000,-0.330520,0.010744,-0.313343,-0.238590
OIL,-0.174315,-0.430414,0.466450,0.396217,0.372830,-0.330520,1.000000,-0.033448,0.013124,-0.231822
SI,0.454565,0.061569,-0.052131,0.004746,0.024798,0.010744,-0.033448,1.000000,-0.069644,-0.053362
resid,0.075844,0.341942,0.302608,-0.324996,-0.161129,-0.313343,0.013124,-0.069644,1.000000,0.542635
TERM,0.092356,0.398803,-0.071969,-0.238530,-0.238081,-0.238590,-0.231822,-0.053362,0.542635,1.000000
